<a href="https://colab.research.google.com/github/seadreamjj/danmu-ID/blob/main/%E7%8C%AB%E8%80%B3%E6%BC%AB%E6%92%AD%E5%BC%B9%E5%B9%95id%E8%AF%BB%E5%8F%96%EF%BC%88%E7%9B%B4%E6%8E%A5%E5%87%BA%E8%A1%A8%E7%89%88%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

直接读取表格循环（猫耳）

In [ ]:
import requests
import re
import pandas as pd

# 读取上传的 Excel 文件
file_path = '/content/猫耳在播剧id（跑程序版）.xlsx'
df = pd.read_excel(file_path)

# 假设表格中有一列名为 'URL'，包含剧集链接
urls = df['url']

# 创建新的列来保存 uid 结果和剧名
df['uid_result'] = None
df['drama_name'] = None

# 定义基础的 API URL
drama_url_base = 'http://www.missevan.com/dramaapi/getdrama?drama_id='
sound_url_base = 'http://www.missevan.com/sound/getdm?soundid='

# 遍历每个 URL
for index, drama_url in enumerate(urls):
    try:
        # 提取剧集的 ID（取URL最后的数字部分）
        drama_id = drama_url.split("/")[-1]

        # 构建完整的 API 请求 URL
        url = drama_url_base + drama_id

        # 请求剧集数据
        drama_result = requests.get(url).text

        # 提取剧名（假设剧名在 JSON 的 "name" 字段中）
        name_pattern = re.compile(r'"name":"(.*?)"')  # 正则表达式提取剧名
        drama_name = re.findall(name_pattern, drama_result)
        if drama_name:
            drama_name = drama_name[0]
        else:
            drama_name = "Unknown"

        # 使用正则表达式提取 sound_id 和 pay_type
        pattern1 = re.compile(r'"sound_id":(\d+),')  # 提取 sound_id
        pattern2 = re.compile(r'"need_pay":(\d+),')  # 提取付费状态 need_pay
        sound_id = re.findall(pattern1, drama_result)
        pay_type = re.findall(pattern2, drama_result)

        # 去掉第一个非付费项
        pay_type = pay_type[1:]

        # 创建 DataFrame 保存提取的数据
        drama = pd.DataFrame({'sound_id': sound_id, 'pay_type': pay_type})

        # 筛选付费 sound_id
        sound_id_filtered = list(drama['sound_id'][drama['pay_type'] == '1'])

        # 获取每个 sound_id 对应的 uid
        uids = []
        for sid in sound_id_filtered:
            sound_url = sound_url_base + sid
            sound_result = requests.get(sound_url).text

            # 提取 uid（假设通过 p="..." 来抓取弹幕信息）
            pattern3 = re.compile(r'p="(.+?)"')
            dm = re.findall(pattern3, sound_result)

            # 分割并提取 uid
            dm_temp = pd.DataFrame({'dm': dm})
            dms_split = dm_temp['dm'].str.split('>', expand=True)[0]
            uid = dms_split.str.split(',', expand=True)[6]

            # 去重处理
            unique_uids = uid.unique().tolist()
            uids.extend(unique_uids)

        # 保存结果到表格中
        df.at[index, 'uid_result'] = ','.join(uids)
        df.at[index, 'drama_name'] = drama_name

        # 打印剧名和 uid 结果
        print(f"剧名: {drama_name}, UIDs: {uids}")

    except Exception as e:
        # 如果发生错误，记录错误信息
        df.at[index, 'uid_result'] = f"Error: {e}"
        df.at[index, 'drama_name'] = "Unknown"
        print(f"处理 URL {drama_url} 时发生错误: {e}")

# 保存结果回到 Excel 文件中
output_path = '/content/猫耳在播剧id（跑程序版）.xlsx'
df.to_excel(output_path, index=False)

print("处理完成，结果已保存到处理后的 Excel 文件中。")


处理 URL 76557 时发生错误: name 'drama_id' is not defined
处理 URL 77771 时发生错误: name 'drama_id' is not defined
处理 URL 68664 时发生错误: name 'drama_id' is not defined
处理 URL 76879 时发生错误: name 'drama_id' is not defined
处理 URL 75856 时发生错误: name 'drama_id' is not defined
处理 URL 77888 时发生错误: name 'drama_id' is not defined
处理 URL 77223 时发生错误: name 'drama_id' is not defined
处理 URL 77249 时发生错误: name 'drama_id' is not defined
处理 URL 77591 时发生错误: name 'drama_id' is not defined
处理 URL 77685 时发生错误: name 'drama_id' is not defined
处理 URL 77035 时发生错误: name 'drama_id' is not defined
处理 URL 75085 时发生错误: name 'drama_id' is not defined
处理 URL 77918 时发生错误: name 'drama_id' is not defined
处理 URL 77937 时发生错误: name 'drama_id' is not defined
处理完成，结果已保存到处理后的 Excel 文件中。


检查api获取是否正确

In [2]:
import pandas as pd

# 读取上传的 Excel 文件
file_path = '/content/猫耳在播剧id（跑程序版）.xlsx'
df = pd.read_excel(file_path)

# 假设表格中有一列名为 'URL'，包含剧集链接
urls = df['url']

# 定义基础的 API URL
drama_url_base = 'http://www.missevan.com/dramaapi/getdrama?drama_id='

# 遍历前几个 URL，输出 drama_id 和 API URL
for index, drama_url in enumerate(urls[:5]):  # 只检查前 5 个 URL
    try:
        # 确保 drama_url 是字符串类型
        if not isinstance(drama_url, str):
            drama_url = str(drama_url)

        # 提取剧集的 ID（取URL最后的数字部分）
        drama_id = drama_url.split("/")[-1]

        # 构建完整的 API 请求 URL
        api_url = drama_url_base + drama_id

        # 打印 drama_id 和 API 请求 URL
        print(f"剧集 ID: {drama_id}")
        print(f"API 请求 URL: {api_url}")
        print("-" * 50)

    except Exception as e:
        print(f"处理 URL {drama_url} 时发生错误: {e}")



剧集 ID: 76557
API 请求 URL: http://www.missevan.com/dramaapi/getdrama?drama_id=76557
--------------------------------------------------
剧集 ID: 77771
API 请求 URL: http://www.missevan.com/dramaapi/getdrama?drama_id=77771
--------------------------------------------------
剧集 ID: 68664
API 请求 URL: http://www.missevan.com/dramaapi/getdrama?drama_id=68664
--------------------------------------------------
剧集 ID: 76879
API 请求 URL: http://www.missevan.com/dramaapi/getdrama?drama_id=76879
--------------------------------------------------
剧集 ID: 75856
API 请求 URL: http://www.missevan.com/dramaapi/getdrama?drama_id=75856
--------------------------------------------------


api没有错误，试试直接输出

In [3]:
import requests
import re
import pandas as pd

# 读取上传的 Excel 文件
file_path = '/content/猫耳在播剧id（跑程序版）.xlsx'
df = pd.read_excel(file_path)

# 假设表格中有 '剧名' 和 'URL' 两列，分别包含剧集名称和剧集链接
titles = df['剧名']
urls = df['url']

# 定义基础的 API URL
drama_url_base = 'http://www.missevan.com/dramaapi/getdrama?drama_id='

# 用于存储最终的结果
final_results = []

# 遍历每个剧集的名称和 URL
for title, drama_url in zip(titles, urls):
    try:
        # 确保 drama_url 是字符串类型
        if not isinstance(drama_url, str):
            drama_url = str(drama_url)

        # 提取剧集的 ID（取URL最后的数字部分）
        drama_id = drama_url.split("/")[-1]

        # 构建完整的 API 请求 URL
        api_url = drama_url_base + drama_id

        # 请求 API 获取剧集数据
        drama_result = requests.get(api_url).text

        # 提取 sound_id 和付费状态
        pattern1 = re.compile(r'"sound_id":(\d+),')  # 提取 sound_id
        pattern2 = re.compile(r'"need_pay":(\d+),')  # 提取付费状态 need_pay
        sound_id = re.findall(pattern1, drama_result)
        pay_type = re.findall(pattern2, drama_result)

        # 去掉第一个非付费项
        pay_type = pay_type[1:]

        # 创建 DataFrame 并筛选付费 sound_id
        drama = pd.DataFrame({'sound_id': sound_id, 'pay_type': pay_type})
        paid_sound_ids = list(drama['sound_id'][drama['pay_type'] == '1'])

        # 存储弹幕 UID 的 DataFrame
        dm1 = pd.DataFrame()

        # 循环处理每个付费 sound_id，获取弹幕信息
        for sid in paid_sound_ids:
            sound_url = f'http://www.missevan.com/sound/getdm?soundid={sid}'
            sound_result = requests.get(sound_url).text

            # 提取弹幕内容
            pattern3 = re.compile(r'p="(.+?)"')
            dm = re.findall(pattern3, sound_result)

            # 将弹幕数据存储到 DataFrame
            dm_temp = pd.DataFrame({'dm': dm})
            dms_split = dm_temp['dm'].str.split('>', expand=True)[0]
            uid = dms_split.str.split(',', expand=True)[6]
            dm_temp['uid'] = uid

            # 合并 UID 数据
            if dm1.empty:
                dm1 = dm_temp
            else:
                dm1 = pd.concat([dm1, dm_temp], ignore_index=True)

        # 输出剧名和不重复的 UID 数量
        unique_uids = len(dm1['uid'].unique())
        print(f"剧名: {title}, 不重复的UID数量: {unique_uids}")

        # 存储结果
        final_results.append({'剧名': title, 'UID数量': unique_uids})

    except Exception as e:
        print(f"处理剧名 {title} 时发生错误: {e}")

# 将结果转为 DataFrame 并导出
results_df = pd.DataFrame(final_results)
print(results_df)


剧名: 惊封2下, 不重复的UID数量: 9777
剧名: 提灯看刺刀（上）, 不重复的UID数量: 5758
剧名: 台风眼2, 不重复的UID数量: 5540
剧名: 吞海3下, 不重复的UID数量: 4323
剧名: 梦魇直播间1, 不重复的UID数量: 3451
剧名: 三嫁2, 不重复的UID数量: 1986
剧名: 跨界2, 不重复的UID数量: 1776
剧名: 暗癖, 不重复的UID数量: 1641
剧名: 偷风不偷月（下）, 不重复的UID数量: 1200
剧名: 千杯, 不重复的UID数量: 829
剧名: 于青, 不重复的UID数量: 743
剧名: 月下安途, 不重复的UID数量: 668
剧名: 三伏2, 不重复的UID数量: 591
剧名: 烟花过境（BG）, 不重复的UID数量: 116
          剧名  UID数量
0       惊封2下   9777
1   提灯看刺刀（上）   5758
2       台风眼2   5540
3       吞海3下   4323
4     梦魇直播间1   3451
5        三嫁2   1986
6        跨界2   1776
7         暗癖   1641
8   偷风不偷月（下）   1200
9         千杯    829
10        于青    743
11      月下安途    668
12       三伏2    591
13  烟花过境（BG）    116


进一步做输出表格版


In [5]:
import requests
import re
import pandas as pd

# 读取上传的 Excel 文件
file_path = '/content/猫耳在播剧id（跑程序版）.xlsx'
df = pd.read_excel(file_path)

# 假设表格中有 '剧名' 和 'URL' 两列，分别包含剧集名称和剧集链接
titles = df['剧名']
urls = df['url']

# 定义基础的 API URL
drama_url_base = 'http://www.missevan.com/dramaapi/getdrama?drama_id='

# 用于存储最终的结果
final_results = []

# 为 DataFrame 添加一列 'id2'，初始为 None
df['id2'] = None

# 遍历每个剧集的名称和 URL
for idx, (title, drama_url) in enumerate(zip(titles, urls)):
    try:
        # 确保 drama_url 是字符串类型
        if not isinstance(drama_url, str):
            drama_url = str(drama_url)

        # 提取剧集的 ID（取URL最后的数字部分）
        drama_id = drama_url.split("/")[-1]

        # 构建完整的 API 请求 URL
        api_url = drama_url_base + drama_id

        # 请求 API 获取剧集数据
        drama_result = requests.get(api_url).text

        # 提取 sound_id 和付费状态
        pattern1 = re.compile(r'"sound_id":(\d+),')  # 提取 sound_id
        pattern2 = re.compile(r'"need_pay":(\d+),')  # 提取付费状态 need_pay
        sound_id = re.findall(pattern1, drama_result)
        pay_type = re.findall(pattern2, drama_result)

        # 去掉第一个非付费项
        pay_type = pay_type[1:]

        # 创建 DataFrame 并筛选付费 sound_id
        drama = pd.DataFrame({'sound_id': sound_id, 'pay_type': pay_type})
        paid_sound_ids = list(drama['sound_id'][drama['pay_type'] == '1'])

        # 存储弹幕 UID 的 DataFrame
        dm1 = pd.DataFrame()

        # 循环处理每个付费 sound_id，获取弹幕信息
        for sid in paid_sound_ids:
            sound_url = f'http://www.missevan.com/sound/getdm?soundid={sid}'
            sound_result = requests.get(sound_url).text

            # 提取弹幕内容
            pattern3 = re.compile(r'p="(.+?)"')
            dm = re.findall(pattern3, sound_result)

            # 将弹幕数据存储到 DataFrame
            dm_temp = pd.DataFrame({'dm': dm})
            dms_split = dm_temp['dm'].str.split('>', expand=True)[0]
            uid = dms_split.str.split(',', expand=True)[6]
            dm_temp['uid'] = uid

            # 合并 UID 数据
            if dm1.empty:
                dm1 = dm_temp
            else:
                dm1 = pd.concat([dm1, dm_temp], ignore_index=True)

        # 获取不重复的 UID 数量
        unique_uids = len(dm1['uid'].unique())

        # 将 UID 结果存储到 DataFrame 中的 'id2' 列
        df.at[idx, 'id2'] = unique_uids

        print(f"剧名: {title}, 不重复的UID数量: {unique_uids}")

    except Exception as e:
        print(f"处理剧名 {title} 时发生错误: {e}")

# 覆盖保存结果到原始 Excel 文件
df.to_excel(file_path, index=False)
print(f"结果已保存到原表格 {file_path}")

剧名: 惊封2下, 不重复的UID数量: 9778
剧名: 提灯看刺刀（上）, 不重复的UID数量: 5758
剧名: 台风眼2, 不重复的UID数量: 5540
剧名: 吞海3下, 不重复的UID数量: 4323
剧名: 梦魇直播间1, 不重复的UID数量: 3451
剧名: 三嫁2, 不重复的UID数量: 1986
剧名: 跨界2, 不重复的UID数量: 1776
剧名: 暗癖, 不重复的UID数量: 1641
剧名: 偷风不偷月（下）, 不重复的UID数量: 1200
剧名: 千杯, 不重复的UID数量: 829
剧名: 于青, 不重复的UID数量: 743
剧名: 月下安途, 不重复的UID数量: 668
剧名: 三伏2, 不重复的UID数量: 591
剧名: 烟花过境（BG）, 不重复的UID数量: 116
结果已保存到原表格 /content/猫耳在播剧id（跑程序版）.xlsx


表格版成功啦！：）

试试漫播版

In [7]:
import requests
import re
import json
import pandas as pd

# 读取上传的 Excel 文件
file_path = '/content/漫播在更剧id（跑程序版）.xlsx'  # 修改为你的文件路径
df = pd.read_excel(file_path)

# 假设表格中有 '剧名' 和 '第一集链接' 两列
titles = df['剧名']
urls = df['第一集链接']

# 为 DataFrame 添加一列 'id2'，初始为 None
df['id2'] = None

# 遍历每个剧集的名称和 '第一集链接'
for idx, (title, url) in enumerate(zip(titles, urls)):
    try:
        # 直接使用链接作为 detail_id
        new_url = url  # 直接使用链接，不再进行拼接

        # 请求新的 URL，获取剧集信息
        r = requests.get(new_url).text
        r = str(json.loads(r)['data']['radioDramaResp']['setRespList'])

        # 使用正则表达式提取集信息
        pattern1 = re.compile("'vipFree': (.+?),")
        vipFree = re.findall(pattern1, r)
        num = len(vipFree)

        pattern2 = re.compile("'setId': (.+?),")
        setId = re.findall(pattern2, r)

        # 找到所有免费集或付费集的 ID
        episodes = [setId[i] for i in range(num) if vipFree[i] == '1']

        # 如果没有免费集，查找付费集
        if len(episodes) == 0:
            pattern3 = re.compile("'payType': (.+?),")
            payType = re.findall(pattern3, r)
            episodes = [setId[i] for i in range(num) if payType[i] == '1']

        # 弹幕列表
        danmu = []

        # 获取每集的弹幕信息
        for each in episodes:
            next_url = "https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=%s&startTime=0&endTime=10000000" % each
            print(f"获取 {title} 的弹幕信息: {next_url}")
            danmakuList = json.loads(requests.get(next_url).content)['b']['danmakuList']

            # 提取每条弹幕的 eid
            for j in danmakuList:
                danmu.append(j['eid'])

        # 获取不重复的 UID 数量
        unique_uids = len(set(danmu))

        # 将 UID 结果存储到 DataFrame 中的 'id2' 列
        df.at[idx, 'id2'] = unique_uids

        print(f"剧名: {title}, 不重复的UID数量: {unique_uids}")

    except Exception as e:
        print(f"处理剧名 {title} 时发生错误: {e}")

# 覆盖保存结果到原始 Excel 文件
df.to_excel(file_path, index=False)
print(f"结果已保存到原表格 {file_path}")


处理剧名 从万米高空降临 时发生错误: Expecting value: line 1 column 1 (char 0)
处理剧名 四面佛 时发生错误: Expecting value: line 1 column 1 (char 0)
处理剧名 臣好柔弱啊 时发生错误: Expecting value: line 1 column 1 (char 0)
处理剧名 青梅狂想曲 时发生错误: Expecting value: line 1 column 1 (char 0)
处理剧名 铜雀锁金钗 时发生错误: Expecting value: line 1 column 1 (char 0)
处理剧名 一枝（第二季） 时发生错误: Expecting value: line 1 column 1 (char 0)
处理剧名 一替成名（上） 时发生错误: Expecting value: line 1 column 1 (char 0)
处理剧名 为了聂先生的恩宠（下） 时发生错误: Expecting value: line 1 column 1 (char 0)
处理剧名 在校生 时发生错误: Expecting value: line 1 column 1 (char 0)
处理剧名 溺酒 时发生错误: Expecting value: line 1 column 1 (char 0)


KeyboardInterrupt: 

检查url是否正确：

In [8]:
import pandas as pd

# 读取上传的 Excel 文件
file_path = '/content/漫播在更剧id（跑程序版）.xlsx'  # 修改为你的文件路径
df = pd.read_excel(file_path)

# 假设表格中有 '剧名' 和 '第一集链接' 两列
titles = df['剧名']
urls = df['第一集链接']

# 遍历每个剧集的名称和 '第一集链接'
for title, url in zip(titles, urls):
    print(f"剧名: {title}, URL: {url}")


剧名: 从万米高空降临, URL: https://www.kilamanbo.com/manbo/pc/detail?id=1944764730950287431&collectId=1966909818551664738
剧名: 四面佛, URL: https://www.kilamanbo.com/manbo/pc/detail?id=1946295508834189409&collectId=1967116320579256664
剧名: 臣好柔弱啊, URL: https://www.kilamanbo.com/manbo/pc/detail?id=1963615668829945940&collectId=1975452822036021343
剧名: 青梅狂想曲, URL: https://www.kilamanbo.com/manbo/pc/detail?id=1955860104649637973&collectId=1984633207257038861
剧名: 铜雀锁金钗, URL: https://www.kilamanbo.com/manbo/pc/detail?id=1983255583611945059&collectId=1986677250912682121
剧名: 一枝（第二季）, URL: https://www.kilamanbo.com/manbo/pc/detail?id=1974326518402252851&collectId=1977313328854204479
剧名: 一替成名（上）, URL: https://www.kilamanbo.com/manbo/pc/detail?id=1969189801940746368&collectId=1983219596080971879
剧名: 为了聂先生的恩宠（下）, URL: https://www.kilamanbo.com/manbo/pc/detail?id=1976606682179960932&collectId=1978858795526258885
剧名: 在校生, URL: https://www.kilamanbo.com/manbo/pc/detail?id=1945472541560668226&collectId=1955466406472

继续检查

In [10]:
import requests
import re
import json
import pandas as pd

# 读取上传的 Excel 文件
file_path = '/content/漫播在更剧id（跑程序版）.xlsx'
df = pd.read_excel(file_path)

# 假设表格中有 '剧名' 和 '第一集链接' 两列
titles = df['剧名']
urls = df['第一集链接']

# 用于存储结果
uids = []

# 遍历每个剧名和 '第一集链接'
for title, url in zip(titles, urls):
    try:
        print(f"处理剧名 {title} 中...")

        # 判断链接类型并构建 API 请求的 URL
        if 'pc' in url:
            detail_id = url.split('Id=')[1].split('&')[0]  # 提取 detail_id
            new_url = f"https://manbo.hongdoulive.com/web_manbo/dramaSetDetail?dramaSetId={detail_id}"
            temp = 'data'
        else:
            episode_id = url.split('id=')[1][:19]  # 提取 episode_id
            radio_id = url.split('DramaId=')[1][:19]  # 提取 radio_id
            new_url = f"https://manbo.hongrenshuo.com.cn/api/v207/radio/drama/set/h5/detail?radioDramaSetId={episode_id}&radioDramaId={radio_id}"
            temp = 'b'

        # 发送请求并解析返回结果
        r = requests.get(new_url).text
        r = str(json.loads(r)[temp]['radioDramaResp']['setRespList'])

        # 正则匹配提取付费集信息
        pattern1 = re.compile("'vipFree': (.+?),")
        vipFree = re.findall(pattern1, r)
        num = len(vipFree)

        pattern2 = re.compile("'setId': (.+?),")
        setId = re.findall(pattern2, r)

        # 找到所有免费集或付费集的 ID
        episodes = [setId[i] for i in range(num) if vipFree[i] == '1']

        if len(episodes) == 0:
            pattern3 = re.compile("'payType': (.+?),")
            payType = re.findall(pattern3, r)
            episodes = [setId[i] for i in range(num) if payType[i] == '1']

        # 弹幕列表
        danmu = []

        # 获取每集的弹幕信息
        for each in episodes:
            next_url = f"https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId={each}&startTime=0&endTime=10000000"
            danmakuList = json.loads(requests.get(next_url).content)['b']['danmakuList']

            # 提取每条弹幕的 uid
            for j in danmakuList:
                danmu.append(j['eid'])

        # 去重并获取最终的 UID 数量
        unique_uids = len(set(danmu))
        print(f"{title} 的 UID 数量: {unique_uids}")

        # 将 UID 数量存储到结果列表中
        uids.append(unique_uids)

    except Exception as e:
        print(f"处理剧名 {title} 时发生错误: {str(e)}")
        uids.append(None)

# 将 UID 结果添加到原表格
df['id2'] = uids

# 保存表格为新的 Excel 文件
output_path = '/content/漫播在更剧id（跑程序版）.xlsx'
df.to_excel(output_path, index=False)
print(f"处理完成，结果已保存至 {output_path}")


处理剧名 从万米高空降临 中...
从万米高空降临 的 UID 数量: 7323
处理剧名 四面佛 中...
四面佛 的 UID 数量: 4024
处理剧名 臣好柔弱啊 中...
臣好柔弱啊 的 UID 数量: 744
处理剧名 青梅狂想曲 中...
青梅狂想曲 的 UID 数量: 613
处理剧名 铜雀锁金钗 中...
铜雀锁金钗 的 UID 数量: 760
处理剧名 一枝（第二季） 中...
一枝（第二季） 的 UID 数量: 449
处理剧名 一替成名（上） 中...
一替成名（上） 的 UID 数量: 418
处理剧名 为了聂先生的恩宠（下） 中...
为了聂先生的恩宠（下） 的 UID 数量: 18
处理剧名 在校生 中...
在校生 的 UID 数量: 2509
处理剧名 溺酒 中...
溺酒 的 UID 数量: 2309
处理剧名 我的老攻是只鬼 中...
我的老攻是只鬼 的 UID 数量: 1165
处理剧名 鬓边不是海棠红 中...
鬓边不是海棠红 的 UID 数量: 726


OSError: Cannot save file into a non-existent directory: '/mnt/data'